In [1]:
import pandas as pd
from langdetect import detect
import numpy as np
import altair as alt
import spacy
from multiprocessing.dummy import Pool
from tqdm import tqdm

In [2]:
df_pr = pd.read_csv("/Users/teng/UBCO/mds_labs/block4/542/data/DevGPT/cleaned/pr_total.csv")
df_issue = pd.read_csv("/Users/teng/UBCO/mds_labs/block4/542/data/DevGPT/cleaned/issue_total.csv")
df_discussion = pd.read_csv("/Users/teng/UBCO/mds_labs/block4/542/data/DevGPT/cleaned/discussion_total.csv")


In [48]:
# Clean them up
# Remove nan answer and promot.
df_pr = df_pr[~df_pr["Answer"].isna()]
df_pr = df_pr[~df_pr["Prompt"].isna()]

df_issue = df_issue[~df_issue["Answer"].isna()]
df_issue = df_issue[~df_issue["Prompt"].isna()]

df_discussion = df_discussion[~df_discussion["Answer"].isna()]
df_discussion = df_discussion[~df_discussion["Prompt"].isna()]

In [49]:
# Filter out Python and English
df_pr = df_pr.loc[df_pr["RepoLanguage"] == "Python"]
df_issue = df_issue.loc[df_issue["RepoLanguage"] == "Python"]
df_discussion = df_discussion.loc[df_discussion["RepoLanguage"] == "Python"]


# Detect answer language
df_pr["Language"] = df_pr["Answer"].apply(detect)
df_issue["Language"] = df_issue["Answer"].apply(detect)
df_discussion["Language"] = df_discussion["Answer"].apply(detect)


# Get English only.
df_pr = df_pr.loc[df_pr["Language"] == "en"]
df_issue = df_issue.loc[df_issue["Language"] == "en"]
df_discussion = df_discussion.loc[df_discussion["Language"] == "en"]

In [50]:
print(len(df_pr))
print(len(df_issue))
print(len(df_discussion))


478
287
62


In [51]:
df_pr.columns

Index(['Unnamed: 0', 'Type', 'URL_pr', 'Author', 'RepoName', 'RepoLanguage',
       'Number', 'Title_x', 'Body', 'CreatedAt', 'ClosedAt', 'MergedAt',
       'UpdatedAt', 'State', 'Additions', 'Deletions', 'ChangedFiles',
       'CommitsTotalCount', 'CommitShas', 'CommitSha', 'source_date',
       'URL_chatgptsharing_x', 'Status', 'DateOfConversation', 'DateOfAccess',
       'Title_y', 'NumberOfPrompts', 'TokensOfPrompts', 'TokensOfAnswers',
       'Model', 'HTMLContent', 'URL_chatgptsharing_y', 'MentionedURL',
       'MentionedProperty', 'MentionedAuthor', 'MentionedText',
       'MentionedPath', 'URL_chatgptsharing', 'Prompt', 'Answer', 'ListOfCode',
       'Language'],
      dtype='object')

In [52]:
df_issue.columns

Index(['Unnamed: 0', 'Type', 'URL_issue', 'Author', 'RepoName', 'RepoLanguage',
       'Number', 'Title_x', 'Body', 'CreatedAt', 'ClosedAt', 'UpdatedAt',
       'State', 'source_date', 'URL_chatgptsharing_x', 'Status',
       'DateOfConversation', 'DateOfAccess', 'Title_y', 'NumberOfPrompts',
       'TokensOfPrompts', 'TokensOfAnswers', 'Model', 'HTMLContent',
       'URL_chatgptsharing_y', 'MentionedURL', 'MentionedProperty',
       'MentionedAuthor', 'MentionedText', 'URL_chatgptsharing', 'Prompt',
       'Answer', 'ListOfCode', 'Language'],
      dtype='object')

In [53]:
df_discussion.columns

Index(['Unnamed: 0', 'Type', 'URL_discussion', 'Author', 'RepoName',
       'RepoLanguage', 'Number', 'Title_x', 'Body', 'CreatedAt', 'ClosedAt',
       'UpdatedAt', 'Closed', 'UpvoteCount', 'source_date',
       'URL_chatgptsharing_x', 'Status', 'DateOfConversation', 'DateOfAccess',
       'Title_y', 'NumberOfPrompts', 'TokensOfPrompts', 'TokensOfAnswers',
       'Model', 'HTMLContent', 'URL_chatgptsharing_y', 'MentionedURL',
       'MentionedProperty', 'MentionedAuthor', 'MentionedText',
       'MentionedIsAnswer', 'MentionedUpvoteCount', 'URL_chatgptsharing',
       'Prompt', 'Answer', 'ListOfCode', 'Language'],
      dtype='object')

In [84]:
df_pr2 = df_pr[["RepoName", "URL_chatgptsharing", "Prompt", "Answer"]].drop_duplicates()
df_pr3 = df_pr[["RepoName", "URL_chatgptsharing_x", "Prompt", "Answer"]].drop_duplicates()
df_pr3.rename(columns={"URL_chatgptsharing_x": "URL_chatgptsharing"}, inplace=True)
df_pr4 = df_pr[["RepoName", "URL_chatgptsharing_y", "Prompt", "Answer"]].drop_duplicates()
df_pr4.rename(columns={"URL_chatgptsharing_y": "URL_chatgptsharing"}, inplace=True)

df_issue2 = df_issue[["RepoName", "URL_chatgptsharing", "Prompt", "Answer"]].drop_duplicates()
df_issue3 = df_issue[["RepoName", "URL_chatgptsharing_x", "Prompt", "Answer"]].drop_duplicates()
df_issue3.rename(columns={"URL_chatgptsharing_x": "URL_chatgptsharing"}, inplace=True)
df_issue4 = df_issue[["RepoName", "URL_chatgptsharing_y", "Prompt", "Answer"]].drop_duplicates()
df_issue4.rename(columns={"URL_chatgptsharing_y": "URL_chatgptsharing"}, inplace=True)

df_discussion2 = df_discussion[["RepoName", "URL_chatgptsharing", "Prompt", "Answer"]].drop_duplicates()
df_discussion3 = df_discussion[["RepoName", "URL_chatgptsharing_x", "Prompt", "Answer"]].drop_duplicates()
df_discussion3.rename(columns={"URL_chatgptsharing_x": "URL_chatgptsharing"}, inplace=True)
df_discussion4 = df_discussion[["RepoName", "URL_chatgptsharing_y", "Prompt", "Answer"]].drop_duplicates()
df_discussion4.rename(columns={"URL_chatgptsharing_y": "URL_chatgptsharing"}, inplace=True)


print(len(df_pr2), len(df_pr3), len(df_pr4))
print(len(df_issue2), len(df_issue3), len(df_issue4))
print(len(df_discussion2), len(df_discussion3), len(df_discussion4))


200 200 200
197 204 204
38 39 39


In [86]:
df = pd.concat(
    [
        df_pr2, df_pr3, df_pr4, df_issue2, df_issue3, df_issue4, df_discussion2,
        df_discussion3, df_discussion4
        ]
    )
df.drop_duplicates()

,RepoName,URL_chatgptsharing,Prompt,Answer
7870,paul-gauthier/aider,https://chat.openai.com/share/4555f0ea-1e7b-49...,How can I setup a github action to automatical...,"Sure, I can certainly help you with that. Here..."
7898,chitalian/gptask,https://chat.openai.com/share/902cd378-3ebc-4e...,Give me some test commands for this\n\nimport ...,This Python script is a command-line tool that...
7906,bbelderbos/htmx-demo,https://chat.openai.com/share/c8c101fa-aaae-49...,how to get the first 20 rows from a django model?,"To get the first 20 rows from a Django model, ..."
7907,bbelderbos/htmx-demo,https://chat.openai.com/share/c8c101fa-aaae-49...,"I have this view for infinite scroll, would be...",Certainly! You can refactor the code to use Dj...
7908,bbelderbos/htmx-demo,https://chat.openai.com/share/c8c101fa-aaae-49...,for page 49 this gives:\n\nInternal Server Err...,Certainly! To handle the ValueError when conve...
...,...,...,...,...
310,uuid6/prototypes,https://chat.openai.com/share/97dbf284-129f-42...,Do you have more?,Certainly! Here are a few additional options f...
311,uuid6/prototypes,https://chat.openai.com/share/97dbf284-129f-42...,And more?,Certainly! Here are a few more options for a s...
312,uuid6/prototypes,https://chat.openai.com/share/97dbf284-129f-42...,And more?,Certainly! Here are a few more options for a s...
313,uuid6/prototypes,https://chat.openai.com/share/97dbf284-129f-42...,More.,Certainly! Here are a few more options for a s...


In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1321 entries, 7870 to 313
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   RepoName            1321 non-null   object
 1   URL_chatgptsharing  1321 non-null   object
 2   Prompt              1321 non-null   object
 3   Answer              1321 non-null   object
dtypes: object(4)
memory usage: 51.6+ KB


In [88]:
df.to_csv("/Users/teng/UBCO/mds_labs/block4/542/data/DevGPT/cleaned/combine.csv")

## Interactions

In [104]:
df.head()

,RepoName,URL_chatgptsharing,Prompt,Answer
7870,paul-gauthier/aider,https://chat.openai.com/share/4555f0ea-1e7b-49...,How can I setup a github action to automatical...,"Sure, I can certainly help you with that. Here..."
7898,chitalian/gptask,https://chat.openai.com/share/902cd378-3ebc-4e...,Give me some test commands for this\n\nimport ...,This Python script is a command-line tool that...
7906,bbelderbos/htmx-demo,https://chat.openai.com/share/c8c101fa-aaae-49...,how to get the first 20 rows from a django model?,"To get the first 20 rows from a Django model, ..."
7907,bbelderbos/htmx-demo,https://chat.openai.com/share/c8c101fa-aaae-49...,"I have this view for infinite scroll, would be...",Certainly! You can refactor the code to use Dj...
7908,bbelderbos/htmx-demo,https://chat.openai.com/share/c8c101fa-aaae-49...,for page 49 this gives:\n\nInternal Server Err...,Certainly! To handle the ValueError when conve...


In [89]:
df_repo = df.groupby(by=["RepoName"])
df_repo.head()

,RepoName,URL_chatgptsharing,Prompt,Answer
7870,paul-gauthier/aider,https://chat.openai.com/share/4555f0ea-1e7b-49...,How can I setup a github action to automatical...,"Sure, I can certainly help you with that. Here..."
7898,chitalian/gptask,https://chat.openai.com/share/902cd378-3ebc-4e...,Give me some test commands for this\n\nimport ...,This Python script is a command-line tool that...
7906,bbelderbos/htmx-demo,https://chat.openai.com/share/c8c101fa-aaae-49...,how to get the first 20 rows from a django model?,"To get the first 20 rows from a Django model, ..."
7907,bbelderbos/htmx-demo,https://chat.openai.com/share/c8c101fa-aaae-49...,"I have this view for infinite scroll, would be...",Certainly! You can refactor the code to use Dj...
7908,bbelderbos/htmx-demo,https://chat.openai.com/share/c8c101fa-aaae-49...,for page 49 this gives:\n\nInternal Server Err...,Certainly! To handle the ValueError when conve...
...,...,...,...,...
227,HaroldMitts/VoAIce,https://chat.openai.com/share/262e8d7d-657b-4a...,"output audio of the following sentence;\n\n""Do...","Sure, here is the audio version of your text:C..."
111,sugi-01096/72,https://chat.openai.com/share/e2c50f86-6c14-4f...,import streamlit as st\nimport json\n\n\ndef s...,The code you provided seems to be a simple bul...
131,sugi-01096/72,https://chat.openai.com/share/4cd4bb4a-56f9-4a...,import streamlit as st\nimport json\n\n\ndef s...,The code you provided seems to be a simple bul...
186,Significant-Gravitas/Auto-GPT,https://chat.openai.com/share/aec2922e-dbcc-47...,I am using venv(python module env) on the mac ...,To upgrade the Python version within a virtual...


In [90]:
len(df_repo["RepoName"].unique())

66

In [91]:
# Mean number of interactions per issuedf_url
ls_url_chatgptsharing = [len(ls) for ls in df_repo.aggregate(list)["URL_chatgptsharing"]]
idx_max = np.argmax(ls_url_chatgptsharing)
idx_min = np.argmin(ls_url_chatgptsharing)
print(
    f"Average interactions per pr: {np.mean(ls_url_chatgptsharing)}\n"
    f"Maximum number of interactions: {ls_url_chatgptsharing[idx_max]}\n"
    f"Minimum number of iteractions: {ls_url_chatgptsharing[idx_min]}"
)

Average interactions per pr: 20.015151515151516
Maximum number of interactions: 501
Minimum number of iteractions: 3


In [110]:
df_repo_count = df_repo["URL_chatgptsharing"].aggregate(len).reset_index()
df_repo_count.columns

Index(['RepoName', 'URL_chatgptsharing'], dtype='object')

In [114]:
# Checking histogarm
# df_url_count.hist(column="URL_chatgptsharing")
alt.Chart(df_repo_count).mark_bar().encode(
    x=alt.X("URL_chatgptsharing:N", title="Number of Interactions with ChatGPT"),
    y="count()",
)

alt.Chart(...)

## Average length of Prompt

In [116]:
nlp = spacy.load("en_core_web_sm")

In [128]:
prompts = df["Prompt"].drop_duplicates().to_list()

In [129]:
# Remove punctuation
with Pool() as pool:
    doc = pool.map(nlp, tqdm(prompts, total=len(prompts)))
    

100%|██████████| 426/426 [00:00<00:00, 586019.52it/s]


In [130]:
tokens = []
for d in tqdm(doc):
    token_without_punc = [token for token in d if not token.is_punct]
    tokens.append(token_without_punc)

100%|██████████| 426/426 [00:00<00:00, 48640.86it/s]


In [131]:
# Calculate average length of prompt
ls_len_promot = [len(token) for token in tokens]
idx_max = np.argmax(ls_len_promot)
idx_min = np.argmin(ls_len_promot)
print(
    f"Average length of prompt per pr: {np.mean(ls_len_promot)}\n"
    f"---------------------------\n"
    f"Maximum length of prompt: {prompts[idx_max]}\n"
    f"---------------------------\n"
    f"Minimum length of prompt: {prompts[idx_min]}"
)

Average length of prompt per pr: 67.9225352112676
---------------------------
Maximum length of prompt: ---------------------------------------------------------------------------
InvalidArgumentError                      Traceback (most recent call last)
<ipython-input-33-22ef21b7d160> in <cell line: 10>()
      8 
      9 # Now, you can use the model for prediction
---> 10 prediction = model.predict(img_expanded)
     11 print(f"This digit is probably a {np.argmax(prediction)}")
     12 

1 frames
/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/execute.py in quick_execute(op_name, num_outputs, inputs, attrs, ctx, name)
     50   try:
     51     ctx.ensure_initialized()
---> 52     tensors = pywrap_tfe.TFE_Py_Execute(ctx._handle, device_name, op_name,
     53                                         inputs, attrs, num_outputs)
     54   except core._NotOkStatusException as e:

InvalidArgumentError: Graph execution error:

Detected at node 'sequential/conv2d/BiasAdd' de

## Average length of Answer

In [132]:
answers = df["Answer"].drop_duplicates().to_list()

In [133]:
# Remove punctuation
with Pool() as pool:
    doc = pool.map(nlp, tqdm(answers, total=len(answers)))

100%|██████████| 433/433 [00:00<00:00, 50554.88it/s]


In [134]:
tokens = []
for d in tqdm(doc):
    token_without_punc = [token for token in d if not token.is_punct]
    tokens.append(token_without_punc)

100%|██████████| 433/433 [00:00<00:00, 25712.62it/s]


In [135]:
# Calculate average length of prompt
ls_len_answer = [len(token) for token in tokens]
idx_max = np.argmax(ls_len_answer)
idx_min = np.argmin(ls_len_answer)
print(
    f"Average length of answer per pr: {np.mean(ls_len_answer)}\n"
    f"---------------------------\n"
    f"Maximum length of answer: {answers[idx_max]}\n"
    f"---------------------------\n"
    f"Minimum length of answer: {answers[idx_min]}"
)

Average length of answer per pr: 179.38337182448038
---------------------------
Maximum length of answer: Certainly! Here's an elaboration on the actions mentioned in the Windows response, providing more details about each mouse click and key press:Navigating the GUI:Left-clicking:Single left-click: Pressing the left mouse button once to select or activate an element, such as buttons, checkboxes, links, or menu items.Right-clicking:Single right-click: Pressing the right mouse button once to open a context menu, displaying additional options specific to the clicked item.Double-clicking:Double left-click: Rapidly pressing the left mouse button twice, with a short interval between clicks, to open files, launch applications, or perform actions associated with the clicked item.Middle-clicking:Single middle-click: Pressing the middle mouse button (usually the scroll wheel) once, typically used to open links in new tabs in web browsers or close tabs.Hovering:Moving the mouse cursor over an el